In [1]:
import numpy as np
import cv2
import os
import torch as t
import visdom
import time

from torch import nn,optim
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils import data

vis = visdom.Visdom(env = 'grad_FC_txt')



In [17]:
kernel_ = np.array([[1/16,1/8,1/16],[1/8,1/4,1/8],[1/16,1/8,1/16]])

p = np.zeros((28,3,3))
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,1] = 1; p[0] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[0,2] = 1; p[1] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,0] = 1; p[2] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[1,2] = 1; p[3] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,0] = 1; p[4] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,1] = 1; p[5] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,0] = 1;zeros[2,2] = 1; p[6] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[0,2] = 1; p[7] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,0] = 1; p[8] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[1,2] = 1; p[9] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,0] = 1; p[10] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,1] = 1; p[11] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,1] = 1;zeros[2,2] = 1; p[12] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,0] = 1; p[13] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[1,2] = 1; p[14] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,0] = 1; p[15] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,1] = 1; p[16] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[0,2] = 1;zeros[2,2] = 1; p[17] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[1,2] = 1; p[18] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,0] = 1; p[19] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,1] = 1; p[20] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,0] = 1;zeros[2,2] = 1; p[21] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,0] = 1; p[22] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,1] = 1; p[23] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[1,2] = 1;zeros[2,2] = 1; p[24] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,1] = 1; p[25] = zeros
zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,0] = 1;zeros[2,2] = 1; p[26] = zeros

zeros = np.zeros((3,3));zeros[1,1] = -2;zeros[2,1] = 1;zeros[2,2] = 1; p[27] = zeros


In [3]:
#修改之前先做个备份，这就是备份。
img_2grad = np.zeros((28,120,120,3))
var_array_r = np.zeros((1,28))
var_array_g = np.zeros((1,28))
var_array_b = np.zeros((1,28))
var_array1 = np.zeros((1,28))
aver_array_r = np.zeros((1,28))
aver_array_g = np.zeros((1,28))
aver_array_b = np.zeros((1,28))
aver_array1 = np.zeros((1,28))
dir_path = "D:\\newfolder43_1"
save_path = "D:\\newfolder43_1_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        image_gaussion = cv2.filter2D(image,cv2.CV_16S,kernel_)
        for m in range(28):
            img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
            var_array[0,3*m] = img_2grad[m][:,:,0].var()
            var_array[0,3*m+1] = img_2grad[m][:,:,1].var()
            var_array[0,3*m+2] = img_2grad[m][:,:,2].var()
            aver_array[0,3*m] = img_2grad[m][:,:,0].mean()
            aver_array[0,3*m+1] = img_2grad[m][:,:,1].mean()
            aver_array[0,3*m+2] = img_2grad[m][:,:,2].mean()
            var_array1[0,m] = img_2grad[m].var()
            aver_array1[0,m] = img_2grad[m].mean()
            
        var_array_max = var_array.max()
        var_array_min = var_array.min()
        aver_array_max = aver_array.max()
        aver_array_min = aver_array.min()
        var_array_max1 = var_array1.max()
        var_array_min1 = var_array1.min()
        aver_array_max1 = aver_array1.max()
        aver_array_min1 = aver_array1.min()
        
        var_array = np.true_divide((var_array-var_array_min),(var_array_max-var_array_min))
        aver_array = np.true_divide((aver_array-aver_array_min),(aver_array_max-aver_array_min))
        var_array1 = np.true_divide((var_array1-var_array_min1),(var_array_max1-var_array_min1))
        aver_array1 = np.true_divide((aver_array1-aver_array_min1),(aver_array_max1-aver_array_min1))
        
        var_array = np.true_divide((var_array-0.5),0.5) 
        aver_array = np.true_divide((aver_array-0.5),0.5)
        var_array1 = np.true_divide((var_array1-0.5),0.5) 
        aver_array1 = np.true_divide((aver_array1-0.5),0.5)
        result_array = np.hstack((var_array,aver_array,var_array1,aver_array1)).reshape(224) 
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in result_array))
        fw.close()

D:\newfolder43_1\clear_face\picture100_147_.png
D:\newfolder43_1\clear_face\picture100_1595_.png
D:\newfolder43_1\clear_face\picture100_1599_.png
D:\newfolder43_1\clear_face\picture100_1601_.png
D:\newfolder43_1\clear_face\picture100_1614_.png
D:\newfolder43_1\clear_face\picture100_1623_.png
D:\newfolder43_1\clear_face\picture100_1626_.png
D:\newfolder43_1\clear_face\picture100_1648_.png
D:\newfolder43_1\clear_face\picture100_1710_.png
D:\newfolder43_1\clear_face\picture100_1712_.png
D:\newfolder43_1\clear_face\picture100_1714_.png
D:\newfolder43_1\clear_face\picture100_1720_.png
D:\newfolder43_1\clear_face\picture100_1723_.png
D:\newfolder43_1\clear_face\picture100_1726_.png
D:\newfolder43_1\clear_face\picture100_1741_.png
D:\newfolder43_1\clear_face\picture100_1745_.png
D:\newfolder43_1\clear_face\picture100_1751_.png
D:\newfolder43_1\clear_face\picture100_1809_.png
D:\newfolder43_1\clear_face\picture100_1813_.png
D:\newfolder43_1\clear_face\picture100_1815_.png
D:\newfolder43_1\clea

D:\newfolder43_1\clear_face\picture100_4538_.png
D:\newfolder43_1\clear_face\picture100_453_.png
D:\newfolder43_1\clear_face\picture100_4566_.png
D:\newfolder43_1\clear_face\picture100_4568_.png
D:\newfolder43_1\clear_face\picture100_4590_.png
D:\newfolder43_1\clear_face\picture100_4624_.png
D:\newfolder43_1\clear_face\picture100_4626_.png
D:\newfolder43_1\clear_face\picture100_4635_.png
D:\newfolder43_1\clear_face\picture100_4637_.png
D:\newfolder43_1\clear_face\picture100_463_.png
D:\newfolder43_1\clear_face\picture100_4641_.png
D:\newfolder43_1\clear_face\picture100_4643_.png
D:\newfolder43_1\clear_face\picture100_4645_.png
D:\newfolder43_1\clear_face\picture100_4646_.png
D:\newfolder43_1\clear_face\picture100_4670_.png
D:\newfolder43_1\clear_face\picture100_4672_.png
D:\newfolder43_1\clear_face\picture100_4677_.png
D:\newfolder43_1\clear_face\picture100_4679_.png
D:\newfolder43_1\clear_face\picture100_4681_.png
D:\newfolder43_1\clear_face\picture100_4684_.png
D:\newfolder43_1\clear

D:\newfolder43_1\clear_face\picture100_587_.png
D:\newfolder43_1\clear_face\picture100_6009_.png
D:\newfolder43_1\clear_face\picture100_6011_.png
D:\newfolder43_1\clear_face\picture100_6013_.png
D:\newfolder43_1\clear_face\picture100_6014_.png
D:\newfolder43_1\clear_face\picture100_6015_.png
D:\newfolder43_1\clear_face\picture100_6016_.png
D:\newfolder43_1\clear_face\picture100_6017_.png
D:\newfolder43_1\clear_face\picture100_6018_.png
D:\newfolder43_1\clear_face\picture100_6030_.png
D:\newfolder43_1\clear_face\picture100_6031_.png
D:\newfolder43_1\clear_face\picture100_6032_.png
D:\newfolder43_1\clear_face\picture100_6033_.png
D:\newfolder43_1\clear_face\picture100_6034_.png
D:\newfolder43_1\clear_face\picture100_6035_.png
D:\newfolder43_1\clear_face\picture100_6037_.png
D:\newfolder43_1\clear_face\picture100_6038_.png
D:\newfolder43_1\clear_face\picture100_6039_.png
D:\newfolder43_1\clear_face\picture100_6040_.png
D:\newfolder43_1\clear_face\picture100_6042_.png
D:\newfolder43_1\clea

D:\newfolder43_1\clear_face\picture1_11376_.png
D:\newfolder43_1\clear_face\picture1_11378_.png
D:\newfolder43_1\clear_face\picture1_1137_.png
D:\newfolder43_1\clear_face\picture1_11380_.png
D:\newfolder43_1\clear_face\picture1_11382_.png
D:\newfolder43_1\clear_face\picture1_11391_.png
D:\newfolder43_1\clear_face\picture1_1139_.png
D:\newfolder43_1\clear_face\picture1_11421_.png
D:\newfolder43_1\clear_face\picture1_11422_.png
D:\newfolder43_1\clear_face\picture1_11424_.png
D:\newfolder43_1\clear_face\picture1_11427_.png
D:\newfolder43_1\clear_face\picture1_11428_.png
D:\newfolder43_1\clear_face\picture1_1142_.png
D:\newfolder43_1\clear_face\picture1_11430_.png
D:\newfolder43_1\clear_face\picture1_11431_.png
D:\newfolder43_1\clear_face\picture1_11434_.png
D:\newfolder43_1\clear_face\picture1_11437_.png
D:\newfolder43_1\clear_face\picture1_11439_.png
D:\newfolder43_1\clear_face\picture1_11442_.png
D:\newfolder43_1\clear_face\picture1_11447_.png
D:\newfolder43_1\clear_face\picture1_11452_

D:\newfolder43_1\clear_face\picture1_1432_.png
D:\newfolder43_1\clear_face\picture1_14713_.png
D:\newfolder43_1\clear_face\picture1_14716_.png
D:\newfolder43_1\clear_face\picture1_14718_.png
D:\newfolder43_1\clear_face\picture1_14721_.png
D:\newfolder43_1\clear_face\picture1_14725_.png
D:\newfolder43_1\clear_face\picture1_14727_.png
D:\newfolder43_1\clear_face\picture1_14729_.png
D:\newfolder43_1\clear_face\picture1_14732_.png
D:\newfolder43_1\clear_face\picture1_14735_.png
D:\newfolder43_1\clear_face\picture1_14737_.png
D:\newfolder43_1\clear_face\picture1_14739_.png
D:\newfolder43_1\clear_face\picture1_14741_.png
D:\newfolder43_1\clear_face\picture1_15138_.png
D:\newfolder43_1\clear_face\picture1_152_.png
D:\newfolder43_1\clear_face\picture1_153_.png
D:\newfolder43_1\clear_face\picture1_1541_.png
D:\newfolder43_1\clear_face\picture1_1544_.png
D:\newfolder43_1\clear_face\picture1_1550_.png
D:\newfolder43_1\clear_face\picture1_1555_.png
D:\newfolder43_1\clear_face\picture1_1563_.png
D:

...

D:\newfolder43_1\fuzzy_face\picture_6255_.png
D:\newfolder43_1\fuzzy_face\picture_6256_.png
D:\newfolder43_1\fuzzy_face\picture_6258_.png
D:\newfolder43_1\fuzzy_face\picture_6259_.png
D:\newfolder43_1\fuzzy_face\picture_6262_.png
D:\newfolder43_1\fuzzy_face\picture_6263_.png
D:\newfolder43_1\fuzzy_face\picture_6264_.png
D:\newfolder43_1\fuzzy_face\picture_6270_.png
D:\newfolder43_1\fuzzy_face\picture_6271_.png
D:\newfolder43_1\fuzzy_face\picture_6273_.png
D:\newfolder43_1\fuzzy_face\picture_6276_.png
D:\newfolder43_1\fuzzy_face\picture_6283_.png
D:\newfolder43_1\fuzzy_face\picture_6287_.png
D:\newfolder43_1\fuzzy_face\picture_6289_.png
D:\newfolder43_1\fuzzy_face\picture_6291_.png
D:\newfolder43_1\fuzzy_face\picture_6316_.png
D:\newfolder43_1\fuzzy_face\picture_6321_.png
D:\newfolder43_1\fuzzy_face\picture_6323_.png
D:\newfolder43_1\fuzzy_face\picture_6324_.png
D:\newfolder43_1\fuzzy_face\picture_6329_.png
D:\newfolder43_1\fuzzy_face\picture_6333_.png
D:\newfolder43_1\fuzzy_face\pictur

D:\newfolder43_1\fuzzy_face\picture_7221_.png
D:\newfolder43_1\fuzzy_face\picture_7222_.png
D:\newfolder43_1\fuzzy_face\picture_7223_.png
D:\newfolder43_1\fuzzy_face\picture_7224_.png
D:\newfolder43_1\fuzzy_face\picture_7227_.png
D:\newfolder43_1\fuzzy_face\picture_722_.png
D:\newfolder43_1\fuzzy_face\picture_7231_.png
D:\newfolder43_1\fuzzy_face\picture_7234_.png
D:\newfolder43_1\fuzzy_face\picture_7238_.png
D:\newfolder43_1\fuzzy_face\picture_7239_.png
D:\newfolder43_1\fuzzy_face\picture_7240_.png
D:\newfolder43_1\fuzzy_face\picture_7241_.png
D:\newfolder43_1\fuzzy_face\picture_7242_.png
D:\newfolder43_1\fuzzy_face\picture_7244_.png
D:\newfolder43_1\fuzzy_face\picture_7246_.png
D:\newfolder43_1\fuzzy_face\picture_7247_.png
D:\newfolder43_1\fuzzy_face\picture_724_.png
D:\newfolder43_1\fuzzy_face\picture_725_.png
D:\newfolder43_1\fuzzy_face\picture_726_.png
D:\newfolder43_1\fuzzy_face\picture_7279_.png
D:\newfolder43_1\fuzzy_face\picture_7280_.png
D:\newfolder43_1\fuzzy_face\picture_72

D:\newfolder43_1\fuzzy_face\picture_7704_.png
D:\newfolder43_1\fuzzy_face\picture_7707_.png
D:\newfolder43_1\fuzzy_face\picture_7710_.png
D:\newfolder43_1\fuzzy_face\picture_7711_.png
D:\newfolder43_1\fuzzy_face\picture_7715_.png
D:\newfolder43_1\fuzzy_face\picture_7716_.png
D:\newfolder43_1\fuzzy_face\picture_7717_.png
D:\newfolder43_1\fuzzy_face\picture_7722_.png
D:\newfolder43_1\fuzzy_face\picture_7727_.png
D:\newfolder43_1\fuzzy_face\picture_7728_.png
D:\newfolder43_1\fuzzy_face\picture_7729_.png
D:\newfolder43_1\fuzzy_face\picture_7733_.png
D:\newfolder43_1\fuzzy_face\picture_7740_.png
D:\newfolder43_1\fuzzy_face\picture_7741_.png
D:\newfolder43_1\fuzzy_face\picture_7743_.png
D:\newfolder43_1\fuzzy_face\picture_7749_.png
D:\newfolder43_1\fuzzy_face\picture_7750_.png
D:\newfolder43_1\fuzzy_face\picture_7751_.png
D:\newfolder43_1\fuzzy_face\picture_7752_.png
D:\newfolder43_1\fuzzy_face\picture_7759_.png
D:\newfolder43_1\fuzzy_face\picture_7760_.png
D:\newfolder43_1\fuzzy_face\pictur

D:\newfolder43_1\fuzzy_face\picture_918_.png
D:\newfolder43_1\fuzzy_face\picture_920_.png
D:\newfolder43_1\fuzzy_face\picture_9218_.png
D:\newfolder43_1\fuzzy_face\picture_9220_.png
D:\newfolder43_1\fuzzy_face\picture_9222_.png
D:\newfolder43_1\fuzzy_face\picture_9225_.png
D:\newfolder43_1\fuzzy_face\picture_9226_.png
D:\newfolder43_1\fuzzy_face\picture_9228_.png
D:\newfolder43_1\fuzzy_face\picture_9230_.png
D:\newfolder43_1\fuzzy_face\picture_9231_.png
D:\newfolder43_1\fuzzy_face\picture_9233_.png
D:\newfolder43_1\fuzzy_face\picture_9234_.png
D:\newfolder43_1\fuzzy_face\picture_9235_.png
D:\newfolder43_1\fuzzy_face\picture_9238_.png
D:\newfolder43_1\fuzzy_face\picture_9239_.png
D:\newfolder43_1\fuzzy_face\picture_9243_.png
D:\newfolder43_1\fuzzy_face\picture_9244_.png
D:\newfolder43_1\fuzzy_face\picture_9245_.png
D:\newfolder43_1\fuzzy_face\picture_9246_.png
D:\newfolder43_1\fuzzy_face\picture_9247_.png
D:\newfolder43_1\fuzzy_face\picture_9249_.png
D:\newfolder43_1\fuzzy_face\picture_

D:\newfolder43_1\fuzzy_face\picture_9842_.png
D:\newfolder43_1\fuzzy_face\picture_9844_.png
D:\newfolder43_1\fuzzy_face\picture_9846_.png
D:\newfolder43_1\fuzzy_face\picture_9849_.png
D:\newfolder43_1\fuzzy_face\picture_9851_.png
D:\newfolder43_1\fuzzy_face\picture_9855_.png
D:\newfolder43_1\fuzzy_face\picture_9856_.png
D:\newfolder43_1\fuzzy_face\picture_9859_.png
D:\newfolder43_1\fuzzy_face\picture_9861_.png
D:\newfolder43_1\fuzzy_face\picture_9863_.png
D:\newfolder43_1\fuzzy_face\picture_9865_.png
D:\newfolder43_1\fuzzy_face\picture_9868_.png
D:\newfolder43_1\fuzzy_face\picture_9871_.png
D:\newfolder43_1\fuzzy_face\picture_9873_.png
D:\newfolder43_1\fuzzy_face\picture_9875_.png
D:\newfolder43_1\fuzzy_face\picture_9878_.png
D:\newfolder43_1\fuzzy_face\picture_987_.png
D:\newfolder43_1\fuzzy_face\picture_9882_.png
D:\newfolder43_1\fuzzy_face\picture_9884_.png
D:\newfolder43_1\fuzzy_face\picture_9888_.png
D:\newfolder43_1\fuzzy_face\picture_988_.png
D:\newfolder43_1\fuzzy_face\picture_

In [18]:
img_2grad = np.zeros((28,30,30,3))
var_array_r = np.zeros((28))
var_array_g = np.zeros((28))
var_array_b = np.zeros((28))
var_array1 = np.zeros((28))
aver_array_r = np.zeros((28))
aver_array_g = np.zeros((28))
aver_array_b = np.zeros((28))
aver_array1 = np.zeros((28))
dir_path = "D:\\newfolder43_1"
save_path = "C:\\newfolder43_1_txt1"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array_r = np.array([])
        all_var_array_g = np.array([])
        all_var_array_b = np.array([])
        all_aver_array_r = np.array([])
        all_aver_array_g = np.array([])
        all_aver_array_b = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array_r[m] = img_2grad[m][:,:,0].std()
                    var_array_g[m] = img_2grad[m][:,:,1].std()
                    var_array_b[m] = img_2grad[m][:,:,2].std()
                    #aver_array_r[m] = img_2grad[m][:,:,0].mean()
                    #aver_array_g[m] = img_2grad[m][:,:,1].mean()
                    #aver_array_b[m] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    #aver_array1[m] = img_2grad[m].mean()
                
                all_var_array_r = np.hstack((all_var_array_r,var_array_r))
                all_var_array_g = np.hstack((all_var_array_g,var_array_g))
                all_var_array_b = np.hstack((all_var_array_b,var_array_b))
                #all_aver_array_r = np.hstack((all_aver_array_r,aver_array_r))
                #all_aver_array_g = np.hstack((all_aver_array_g,aver_array_g))
                #all_aver_array_b = np.hstack((all_aver_array_b,aver_array_b))
                all_var_array1 = np.hstack((all_var_array1,var_array1))
               # all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        all_var_array_r_max = all_var_array_r.max()
        all_var_array_r_min = all_var_array_r.min()
        all_var_array_g_max = all_var_array_g.max()
        all_var_array_g_min = all_var_array_g.min()
        all_var_array_b_max = all_var_array_b.max()
        all_var_array_b_min = all_var_array_b.min()
        
        #all_aver_array_r_max = all_aver_array_r.max()
        #all_aver_array_r_min = all_aver_array_r.min()
        #all_aver_array_g_max = all_aver_array_g.max()
        #all_aver_array_g_min = all_aver_array_g.min()
        #all_aver_array_b_max = all_aver_array_b.max()
        #all_aver_array_b_min = all_aver_array_b.min()
        
        all_var_array_max1 = all_var_array1.max()
        all_var_array_min1 = all_var_array1.min()
        #all_aver_array_max1 = all_aver_array1.max()
        #all_aver_array_min1 = all_aver_array1.min()
        
        all_var_array_r = np.true_divide((all_var_array_r-all_var_array_r_min),(all_var_array_r_max-all_var_array_r_min))
        all_var_array_g = np.true_divide((all_var_array_g-all_var_array_g_min),(all_var_array_g_max-all_var_array_g_min))
        all_var_array_b = np.true_divide((all_var_array_b-all_var_array_b_min),(all_var_array_b_max-all_var_array_b_min))
        #all_aver_array_r = np.true_divide((all_aver_array_r-all_aver_array_r_min),(all_aver_array_r_max-all_aver_array_r_min))
        #all_aver_array_g = np.true_divide((all_aver_array_g-all_aver_array_g_min),(all_aver_array_g_max-all_aver_array_g_min))
        #all_aver_array_b = np.true_divide((all_aver_array_b-all_aver_array_b_min),(all_aver_array_b_max-all_aver_array_b_min))
        all_var_array1 = np.true_divide((all_var_array1-all_var_array_min1),(all_var_array_max1-all_var_array_min1))
        #all_aver_array1 = np.true_divide((all_aver_array1-all_aver_array_min1),(all_aver_array_max1-all_aver_array_min1)) 
        
        all_var_array_r = np.true_divide((all_var_array_r-0.5),0.5) 
        all_var_array_g = np.true_divide((all_var_array_g-0.5),0.5) 
        all_var_array_b = np.true_divide((all_var_array_b-0.5),0.5) 
        #all_aver_array_r = np.true_divide((all_aver_array_r-0.5),0.5)
        #all_aver_array_g = np.true_divide((all_aver_array_g-0.5),0.5)
        #all_aver_array_b = np.true_divide((all_aver_array_b-0.5),0.5)
        all_var_array1 = np.true_divide((all_var_array1-0.5),0.5) 
        #all_aver_array1 = np.true_divide((all_aver_array1-0.5),0.5) 
                
        #all_result_array = np.hstack((all_var_array_r,all_var_array_g,all_var_array_b,all_aver_array_r,all_aver_array_g,all_aver_array_b,all_var_array1,all_aver_array1))       
        all_result_array = np.hstack((all_var_array_r,all_var_array_g,all_var_array_b,all_var_array1))
        
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in all_result_array))
        fw.close()

D:\newfolder43_1\clear_face\picture100_147_.png
D:\newfolder43_1\clear_face\picture100_1595_.png
D:\newfolder43_1\clear_face\picture100_1599_.png
D:\newfolder43_1\clear_face\picture100_1601_.png
D:\newfolder43_1\clear_face\picture100_1614_.png
D:\newfolder43_1\clear_face\picture100_1623_.png
D:\newfolder43_1\clear_face\picture100_1626_.png
D:\newfolder43_1\clear_face\picture100_1648_.png
D:\newfolder43_1\clear_face\picture100_1710_.png
D:\newfolder43_1\clear_face\picture100_1712_.png
D:\newfolder43_1\clear_face\picture100_1714_.png
D:\newfolder43_1\clear_face\picture100_1720_.png
D:\newfolder43_1\clear_face\picture100_1723_.png
D:\newfolder43_1\clear_face\picture100_1726_.png
D:\newfolder43_1\clear_face\picture100_1741_.png
D:\newfolder43_1\clear_face\picture100_1745_.png
D:\newfolder43_1\clear_face\picture100_1751_.png
D:\newfolder43_1\clear_face\picture100_1809_.png
D:\newfolder43_1\clear_face\picture100_1813_.png
D:\newfolder43_1\clear_face\picture100_1815_.png
D:\newfolder43_1\clea

D:\newfolder43_1\clear_face\picture100_4534_.png
D:\newfolder43_1\clear_face\picture100_4538_.png
D:\newfolder43_1\clear_face\picture100_453_.png
D:\newfolder43_1\clear_face\picture100_4566_.png
D:\newfolder43_1\clear_face\picture100_4568_.png
D:\newfolder43_1\clear_face\picture100_4590_.png
D:\newfolder43_1\clear_face\picture100_4624_.png
D:\newfolder43_1\clear_face\picture100_4626_.png
D:\newfolder43_1\clear_face\picture100_4635_.png
D:\newfolder43_1\clear_face\picture100_4637_.png
D:\newfolder43_1\clear_face\picture100_463_.png
D:\newfolder43_1\clear_face\picture100_4641_.png
D:\newfolder43_1\clear_face\picture100_4643_.png
D:\newfolder43_1\clear_face\picture100_4645_.png
D:\newfolder43_1\clear_face\picture100_4646_.png
D:\newfolder43_1\clear_face\picture100_4670_.png
D:\newfolder43_1\clear_face\picture100_4672_.png
D:\newfolder43_1\clear_face\picture100_4677_.png
D:\newfolder43_1\clear_face\picture100_4679_.png
D:\newfolder43_1\clear_face\picture100_4681_.png
D:\newfolder43_1\clear

D:\newfolder43_1\clear_face\picture100_5849_.png
D:\newfolder43_1\clear_face\picture100_584_.png
D:\newfolder43_1\clear_face\picture100_5850_.png
D:\newfolder43_1\clear_face\picture100_5851_.png
D:\newfolder43_1\clear_face\picture100_5852_.png
D:\newfolder43_1\clear_face\picture100_585_.png
D:\newfolder43_1\clear_face\picture100_586_.png
D:\newfolder43_1\clear_face\picture100_587_.png
D:\newfolder43_1\clear_face\picture100_6009_.png
D:\newfolder43_1\clear_face\picture100_6011_.png
D:\newfolder43_1\clear_face\picture100_6013_.png
D:\newfolder43_1\clear_face\picture100_6014_.png
D:\newfolder43_1\clear_face\picture100_6015_.png
D:\newfolder43_1\clear_face\picture100_6016_.png
D:\newfolder43_1\clear_face\picture100_6017_.png
D:\newfolder43_1\clear_face\picture100_6018_.png
D:\newfolder43_1\clear_face\picture100_6030_.png
D:\newfolder43_1\clear_face\picture100_6031_.png
D:\newfolder43_1\clear_face\picture100_6032_.png
D:\newfolder43_1\clear_face\picture100_6033_.png
D:\newfolder43_1\clear_f

D:\newfolder43_1\clear_face\picture1_1134_.png
D:\newfolder43_1\clear_face\picture1_11354_.png
D:\newfolder43_1\clear_face\picture1_11358_.png
D:\newfolder43_1\clear_face\picture1_11360_.png
D:\newfolder43_1\clear_face\picture1_11363_.png
D:\newfolder43_1\clear_face\picture1_11365_.png
D:\newfolder43_1\clear_face\picture1_11368_.png
D:\newfolder43_1\clear_face\picture1_11370_.png
D:\newfolder43_1\clear_face\picture1_11372_.png
D:\newfolder43_1\clear_face\picture1_11374_.png
D:\newfolder43_1\clear_face\picture1_11376_.png
D:\newfolder43_1\clear_face\picture1_11378_.png
D:\newfolder43_1\clear_face\picture1_1137_.png
D:\newfolder43_1\clear_face\picture1_11380_.png
D:\newfolder43_1\clear_face\picture1_11382_.png
D:\newfolder43_1\clear_face\picture1_11391_.png
D:\newfolder43_1\clear_face\picture1_1139_.png
D:\newfolder43_1\clear_face\picture1_11421_.png
D:\newfolder43_1\clear_face\picture1_11422_.png
D:\newfolder43_1\clear_face\picture1_11424_.png
D:\newfolder43_1\clear_face\picture1_11427_

D:\newfolder43_1\clear_face\picture1_1394_.png
D:\newfolder43_1\clear_face\picture1_1398_.png
D:\newfolder43_1\clear_face\picture1_1400_.png
D:\newfolder43_1\clear_face\picture1_1403_.png
D:\newfolder43_1\clear_face\picture1_1407_.png
D:\newfolder43_1\clear_face\picture1_14086_.png
D:\newfolder43_1\clear_face\picture1_1409_.png
D:\newfolder43_1\clear_face\picture1_1420_.png
D:\newfolder43_1\clear_face\picture1_1421_.png
D:\newfolder43_1\clear_face\picture1_1424_.png
D:\newfolder43_1\clear_face\picture1_14280_.png
D:\newfolder43_1\clear_face\picture1_14281_.png
D:\newfolder43_1\clear_face\picture1_14282_.png
D:\newfolder43_1\clear_face\picture1_14283_.png
D:\newfolder43_1\clear_face\picture1_1429_.png
D:\newfolder43_1\clear_face\picture1_1432_.png
D:\newfolder43_1\clear_face\picture1_14713_.png
D:\newfolder43_1\clear_face\picture1_14716_.png
D:\newfolder43_1\clear_face\picture1_14718_.png
D:\newfolder43_1\clear_face\picture1_14721_.png
D:\newfolder43_1\clear_face\picture1_14725_.png
D:\

...

D:\newfolder43_1\fuzzy_face\picture_6593_.png
D:\newfolder43_1\fuzzy_face\picture_661_.png
D:\newfolder43_1\fuzzy_face\picture_662_.png
D:\newfolder43_1\fuzzy_face\picture_663_.png
D:\newfolder43_1\fuzzy_face\picture_664_.png
D:\newfolder43_1\fuzzy_face\picture_6657_.png
D:\newfolder43_1\fuzzy_face\picture_6658_.png
D:\newfolder43_1\fuzzy_face\picture_6661_.png
D:\newfolder43_1\fuzzy_face\picture_6663_.png
D:\newfolder43_1\fuzzy_face\picture_6664_.png
D:\newfolder43_1\fuzzy_face\picture_6665_.png
D:\newfolder43_1\fuzzy_face\picture_6667_.png
D:\newfolder43_1\fuzzy_face\picture_672_.png
D:\newfolder43_1\fuzzy_face\picture_674_.png
D:\newfolder43_1\fuzzy_face\picture_6772_.png
D:\newfolder43_1\fuzzy_face\picture_6774_.png
D:\newfolder43_1\fuzzy_face\picture_6775_.png
D:\newfolder43_1\fuzzy_face\picture_6778_.png
D:\newfolder43_1\fuzzy_face\picture_6794_.png
D:\newfolder43_1\fuzzy_face\picture_6795_.png
D:\newfolder43_1\fuzzy_face\picture_6796_.png
D:\newfolder43_1\fuzzy_face\picture_679_

D:\newfolder43_1\fuzzy_face\picture_7334_.png
D:\newfolder43_1\fuzzy_face\picture_7336_.png
D:\newfolder43_1\fuzzy_face\picture_7338_.png
D:\newfolder43_1\fuzzy_face\picture_7340_.png
D:\newfolder43_1\fuzzy_face\picture_7342_.png
D:\newfolder43_1\fuzzy_face\picture_7343_.png
D:\newfolder43_1\fuzzy_face\picture_7344_.png
D:\newfolder43_1\fuzzy_face\picture_7345_.png
D:\newfolder43_1\fuzzy_face\picture_7347_.png
D:\newfolder43_1\fuzzy_face\picture_7349_.png
D:\newfolder43_1\fuzzy_face\picture_7350_.png
D:\newfolder43_1\fuzzy_face\picture_7351_.png
D:\newfolder43_1\fuzzy_face\picture_7352_.png
D:\newfolder43_1\fuzzy_face\picture_7353_.png
D:\newfolder43_1\fuzzy_face\picture_7354_.png
D:\newfolder43_1\fuzzy_face\picture_7355_.png
D:\newfolder43_1\fuzzy_face\picture_7356_.png
D:\newfolder43_1\fuzzy_face\picture_7358_.png
D:\newfolder43_1\fuzzy_face\picture_7359_.png
D:\newfolder43_1\fuzzy_face\picture_7360_.png
D:\newfolder43_1\fuzzy_face\picture_7361_.png
D:\newfolder43_1\fuzzy_face\pictur

D:\newfolder43_1\fuzzy_face\picture_7795_.png
D:\newfolder43_1\fuzzy_face\picture_7800_.png
D:\newfolder43_1\fuzzy_face\picture_7801_.png
D:\newfolder43_1\fuzzy_face\picture_7829_.png
D:\newfolder43_1\fuzzy_face\picture_7831_.png
D:\newfolder43_1\fuzzy_face\picture_7832_.png
D:\newfolder43_1\fuzzy_face\picture_7834_.png
D:\newfolder43_1\fuzzy_face\picture_7836_.png
D:\newfolder43_1\fuzzy_face\picture_7841_.png
D:\newfolder43_1\fuzzy_face\picture_784_.png
D:\newfolder43_1\fuzzy_face\picture_7880_.png
D:\newfolder43_1\fuzzy_face\picture_7882_.png
D:\newfolder43_1\fuzzy_face\picture_7884_.png
D:\newfolder43_1\fuzzy_face\picture_7974_.png
D:\newfolder43_1\fuzzy_face\picture_7990_.png
D:\newfolder43_1\fuzzy_face\picture_7996_.png
D:\newfolder43_1\fuzzy_face\picture_7999_.png
D:\newfolder43_1\fuzzy_face\picture_7_.png
D:\newfolder43_1\fuzzy_face\picture_8000_.png
D:\newfolder43_1\fuzzy_face\picture_8002_.png
D:\newfolder43_1\fuzzy_face\picture_8003_.png
D:\newfolder43_1\fuzzy_face\picture_80

D:\newfolder43_1\fuzzy_face\picture_9268_.png
D:\newfolder43_1\fuzzy_face\picture_927_.png
D:\newfolder43_1\fuzzy_face\picture_9298_.png
D:\newfolder43_1\fuzzy_face\picture_9299_.png
D:\newfolder43_1\fuzzy_face\picture_9315_.png
D:\newfolder43_1\fuzzy_face\picture_9320_.png
D:\newfolder43_1\fuzzy_face\picture_9326_.png
D:\newfolder43_1\fuzzy_face\picture_9332_.png
D:\newfolder43_1\fuzzy_face\picture_9337_.png
D:\newfolder43_1\fuzzy_face\picture_9342_.png
D:\newfolder43_1\fuzzy_face\picture_9348_.png
D:\newfolder43_1\fuzzy_face\picture_9356_.png
D:\newfolder43_1\fuzzy_face\picture_936_.png
D:\newfolder43_1\fuzzy_face\picture_9374_.png
D:\newfolder43_1\fuzzy_face\picture_938_.png
D:\newfolder43_1\fuzzy_face\picture_9446_.png
D:\newfolder43_1\fuzzy_face\picture_944_.png
D:\newfolder43_1\fuzzy_face\picture_949_.png
D:\newfolder43_1\fuzzy_face\picture_951_.png
D:\newfolder43_1\fuzzy_face\picture_952_.png
D:\newfolder43_1\fuzzy_face\picture_953_.png
D:\newfolder43_1\fuzzy_face\picture_954_.p

D:\newfolder43_1\fuzzy_face\picture_9908_.png
D:\newfolder43_1\fuzzy_face\picture_9911_.png
D:\newfolder43_1\fuzzy_face\picture_9913_.png
D:\newfolder43_1\fuzzy_face\picture_9915_.png
D:\newfolder43_1\fuzzy_face\picture_9918_.png
D:\newfolder43_1\fuzzy_face\picture_991_.png
D:\newfolder43_1\fuzzy_face\picture_9921_.png
D:\newfolder43_1\fuzzy_face\picture_992_.png
D:\newfolder43_1\fuzzy_face\picture_993_.png
D:\newfolder43_1\fuzzy_face\picture_995_.png
D:\newfolder43_1\fuzzy_face\picture_998_.png
D:\newfolder43_1\fuzzy_face\picture_999_.png
D:\newfolder43_1\fuzzy_face\picture_9_.png


In [19]:
#实验CELL
path = os.listdir("D:\\newfolder43_1\\clear_face")
(path[0].split("."))[0]

'picture100_147_'

In [2]:
#网络model
class FC_Net(nn.Module):
    def __init__(self,input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim):
        super(FC_Net,self).__init__()
        self.layer1 = nn.Sequential(
                                    
                                    nn.Linear(input_dim,n_hidden_1),
                                    nn.BatchNorm1d(n_hidden_1),
                                    nn.ReLU(True)
                                    )
        self.layer2 = nn.Sequential(
                                    nn.Linear(n_hidden_1,n_hidden_2),
                                    nn.BatchNorm1d(n_hidden_2),
                                    nn.ReLU(True)
                                    )
        self.layer3 = nn.Sequential(
                                    nn.Linear(n_hidden_2,n_hidden_3),
                                    nn.BatchNorm1d(n_hidden_3),
                                    nn.ReLU(True)
                                    )
        self.layer_out = nn.Sequential(
                                    nn.Linear(n_hidden_3,output_dim),
                                    #nn.BatchNorm1d(output_dim),
                                    #nn.ReLU(True)
                                       )

    def forward(self,x):
        #print("x.shape",x.shape)
        x = self.layer1(x)
        #print("x.shape1",x.shape)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer_out(x)
        #x = self.softmax(x)
        #print(x)
        return x

In [3]:
class txt_dataset(data.Dataset):
    def __init__(self,root,transforms0=None):
        self.paths = [root+"\\"+path+"\\"+val for path in os.listdir(root) for val in os.listdir(root+"\\" +path)]
        self.transforms0 = transforms0
        
    def __getitem__(self,index):   
        data_path = self.paths[index]
        label = 1 if 'clear_face' in data_path.split("\\")[-2] else 0
        fr = open(data_path)
        data_str = fr.read()
        array = np.array(data_str.split(",")).astype(float)
        fr.close()
        data = t.from_numpy(array)        
        
        return data,label
    
    def __len__(self):
        return len(self.paths)
                                        

In [36]:
#实验CELL
#这个CELL极其重要，尤其是最后一行，我用一行就读取了一个文件中所有的文件夹下的所有文件。
root = "D:\\newfolder43"
paths = os.listdir(root)
lists = [os.listdir(root+"\\"+path) for path in paths]
[val for sublist in lists for val in sublist]
[root+"\\"+path+"\\"+val for path in paths for val in os.listdir(root+"\\" +path)]

['D:\\newfolder43\\clear_front_face\\picture70_10020_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10063_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10064_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10066_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10067_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10070_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10072_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10073_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10075_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10076_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10078_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10079_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10080_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10082_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10087_.png',
 'D:\\newfolder43\\clear_front_face\\picture70_10089_.png',
 'D:\\newfolder43\\clear_front_face\\pic

In [18]:
#实验的CELL   
#说明，hstack()可以多个矩阵。
a = np.array([[1,-1,1,1]])
b = np.array([[-1,-1,1,-1]])
c = np.array([[-1,-1,0,-1]])
d = np.array([[0,-1,1,-1]])
np.true_divide((a-a.mean()),a.std())
np.hstack((a,b,c,d))

f = np.array([[[1,-1,2,1]],[[1,-1,1,1]]])
f

array([[[ 1, -1,  2,  1]],

       [[ 1, -1,  1,  1]]])

In [7]:
#实验CELL
a = np.array([])
b = np.array([1])
c = np.hstack((a,b))
c.shape

(1,)

In [14]:
#实验的CELL
dataset1 = ImageFolder("D:\\newfolder43_1")
img = dataset1[0][0]
image = cv2.cvtColor(np.asarray(img),cv2.COLOR_RGB2BGR)
cv2.imshow("OpenCV",image)  
cv2.waitKey() 

-1

In [19]:
#实验的CELL
image = cv2.imread("D:\\newfolder43_1\\clear_face\\picture_3144_.png")
#image = image[:,:,0]
print(image.var())
image = cv2.cvtColor(np.asarray(image),cv2.COLOR_RGB2BGR)
print(image.var())
cv2.imshow("OpenCV",image)  
cv2.waitKey() 

1728.6520611234355
1728.6520611234355


-1

In [26]:
#实验的CELL
normalize = transforms.Normalize(mean=[0.5,0.5,0.5],std=[0.5,0.5,0.5])   #感觉不应该进行方差标准化，因为最终图像就是使用方差来完成模糊程度判断的。
transform0 = transforms.Compose([
                                #transforms.RandomRotation(20),         #图像旋转的角度
                                #transforms.RandomHorizontalFlip(),    #水平方向翻转
                                transforms.ToTensor(),
                                normalize()
                                ])
dataset = ImageFolder("D:\\newfolder43_1",transform=transform0)

TypeError: __call__() missing 1 required positional argument: 'tensor'

In [4]:
batch_size = 64
learning_rate = 1e-4
num_epoches = 500
weight_decay = 1e-4

In [5]:
dataset0 = txt_dataset("C:\\newfolder43_1_txt1")
dataset0[0][1]


1

In [6]:
dataloader = DataLoader(dataset0,batch_size=batch_size,shuffle=True)
#dataiter = iter(dataloader)
#imgs,labels = next(dataiter)
#labels

In [7]:
input_dim=3584
n_hidden_1=128
n_hidden_2=64
n_hidden_3=32
output_dim=2
model = FC_Net(input_dim,n_hidden_1,n_hidden_2,n_hidden_3,output_dim)

In [8]:
if t.cuda.is_available():
    model = model.cuda()
    
criterion = nn.CrossEntropyLoss()
criterion.cuda() 
optimizer = optim.Adam(model.parameters(),lr=learning_rate,weight_decay=weight_decay) 

In [10]:
print("training start time  <===>  ",time.localtime())
sum_loss = t.Tensor([0])
for epoch in range(0,num_epoches):
    for i,data in enumerate(dataloader,0):
        data_batch, label = data        
        data_batch, label = Variable(data_batch.float()).cuda(),Variable(label).cuda()
        output = model(data_batch)
        optimizer.zero_grad()
        loss = criterion(output,label)
        loss.backward() 
        optimizer.step()
        
        sum_loss = t.cat((sum_loss,loss.cpu().data),0)
        x = i+epoch*len(dataloader)
        if x%50 == 49:
            loss_aver = sum_loss.mean()
            sum_loss = t.Tensor([0])
            vis.line(X=t.Tensor([x]), Y=t.Tensor([loss_aver]), win='blur1',opts={'title': 'loss-batch2', 'xlabel':'batch_num','ylabel':'loss_aver'},update='append' if x>49 else None)
            #vis.updateTrace(X=t.Tensor([x]),Y=t.Tensor([loss_aver]),win='face_angle', name='7')
t.save(model.state_dict(),"C:\\Users\\SiChen\\face_sys\\grad_var_full_layers\\model_save\\fc_network_300_128_64_NoSoftmax_3584_rgb_guiyihua_0.5Normalize_txt_lr1e-4.pth") 
print("training start time  <===>  ",time.localtime())

training start time  <===>   time.struct_time(tm_year=2018, tm_mon=6, tm_mday=22, tm_hour=10, tm_min=21, tm_sec=57, tm_wday=4, tm_yday=173, tm_isdst=0)
training start time  <===>   time.struct_time(tm_year=2018, tm_mon=6, tm_mday=22, tm_hour=21, tm_min=19, tm_sec=23, tm_wday=4, tm_yday=173, tm_isdst=0)


#以下是测试部分

In [11]:
from torch.utils import data

In [12]:
img_2grad = np.zeros((28,30,30,3))
var_array = np.zeros((84))
var_array1 = np.zeros((28))
aver_array = np.zeros((84))
aver_array1 = np.zeros((28))
dir_path = "D:\\newfolder43_2"
save_path = "D:\\newfolder43_2_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array = np.array([])
        all_aver_array = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array[3*m] = img_2grad[m][:,:,0].var()
                    var_array[3*m+1] = img_2grad[m][:,:,1].var()
                    var_array[3*m+2] = img_2grad[m][:,:,2].var()
                    aver_array[3*m] = img_2grad[m][:,:,0].mean()
                    aver_array[3*m+1] = img_2grad[m][:,:,1].mean()
                    aver_array[3*m+2] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].var()
                    aver_array1[m] = img_2grad[m].mean()
                
                all_var_array = np.hstack((all_var_array,var_array))
                all_aver_array = np.hstack((all_aver_array,aver_array))   
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        all_var_array_max = all_var_array.max()
        all_var_array_min = all_var_array.min()
        all_aver_array_max = all_aver_array.max()
        all_aver_array_min = all_aver_array.min()
        all_var_array_max1 = all_var_array1.max()
        all_var_array_min1 = all_var_array1.min()
        all_aver_array_max1 = all_aver_array1.max()
        all_aver_array_min1 = all_aver_array1.min()
        
        all_var_array = np.true_divide((all_var_array-all_var_array_min),(all_var_array_max-all_var_array_min))
        all_aver_array = np.true_divide((all_aver_array-all_aver_array_min),(all_aver_array_max-all_aver_array_min))
        all_var_array1 = np.true_divide((all_var_array1-all_var_array_min1),(all_var_array_max1-all_var_array_min1))
        all_aver_array1 = np.true_divide((all_aver_array1-all_aver_array_min1),(all_aver_array_max1-all_aver_array_min1))     
                
        all_result_array = np.hstack((all_var_array,all_aver_array,all_var_array1,all_aver_array1))       
             
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in all_result_array))
        fw.close()

D:\newfolder43_2\clear_face\picture_1000_.png
D:\newfolder43_2\clear_face\picture_1002_.png
D:\newfolder43_2\clear_face\picture_1004_.png
D:\newfolder43_2\clear_face\picture_1005_.png
D:\newfolder43_2\clear_face\picture_1008_.png
D:\newfolder43_2\clear_face\picture_1009_.png
D:\newfolder43_2\clear_face\picture_100_.png
D:\newfolder43_2\clear_face\picture_1010_.png
D:\newfolder43_2\clear_face\picture_1013_.png
D:\newfolder43_2\clear_face\picture_1014_.png
D:\newfolder43_2\clear_face\picture_1015_.png
D:\newfolder43_2\clear_face\picture_1016_.png
D:\newfolder43_2\clear_face\picture_1018_.png
D:\newfolder43_2\clear_face\picture_1019_.png
D:\newfolder43_2\clear_face\picture_101_.png
D:\newfolder43_2\clear_face\picture_1020_.png
D:\newfolder43_2\clear_face\picture_1021_.png
D:\newfolder43_2\clear_face\picture_1024_.png
D:\newfolder43_2\clear_face\picture_1025_.png
D:\newfolder43_2\clear_face\picture_1027_.png
D:\newfolder43_2\clear_face\picture_1028_.png
D:\newfolder43_2\clear_face\picture_

D:\newfolder43_2\clear_face\picture_1322_.png
D:\newfolder43_2\clear_face\picture_1326_.png
D:\newfolder43_2\clear_face\picture_1327_.png
D:\newfolder43_2\clear_face\picture_1328_.png
D:\newfolder43_2\clear_face\picture_1329_.png
D:\newfolder43_2\clear_face\picture_132_.png
D:\newfolder43_2\clear_face\picture_1330_.png
D:\newfolder43_2\clear_face\picture_1331_.png
D:\newfolder43_2\clear_face\picture_1332_.png
D:\newfolder43_2\clear_face\picture_1333_.png
D:\newfolder43_2\clear_face\picture_1334_.png
D:\newfolder43_2\clear_face\picture_1335_.png
D:\newfolder43_2\clear_face\picture_1337_.png
D:\newfolder43_2\clear_face\picture_1338_.png
D:\newfolder43_2\clear_face\picture_1339_.png
D:\newfolder43_2\clear_face\picture_1340_.png
D:\newfolder43_2\clear_face\picture_1341_.png
D:\newfolder43_2\clear_face\picture_1342_.png
D:\newfolder43_2\clear_face\picture_1343_.png
D:\newfolder43_2\clear_face\picture_1344_.png
D:\newfolder43_2\clear_face\picture_1345_.png
D:\newfolder43_2\clear_face\picture

D:\newfolder43_2\clear_face\picture_165_.png
D:\newfolder43_2\clear_face\picture_1661_.png
D:\newfolder43_2\clear_face\picture_1663_.png
D:\newfolder43_2\clear_face\picture_1664_.png
D:\newfolder43_2\clear_face\picture_1666_.png
D:\newfolder43_2\clear_face\picture_1668_.png
D:\newfolder43_2\clear_face\picture_166_.png
D:\newfolder43_2\clear_face\picture_1671_.png
D:\newfolder43_2\clear_face\picture_1673_.png
D:\newfolder43_2\clear_face\picture_1676_.png
D:\newfolder43_2\clear_face\picture_167_.png
D:\newfolder43_2\clear_face\picture_1683_.png
D:\newfolder43_2\clear_face\picture_1684_.png
D:\newfolder43_2\clear_face\picture_1685_.png
D:\newfolder43_2\clear_face\picture_1686_.png
D:\newfolder43_2\clear_face\picture_1688_.png
D:\newfolder43_2\clear_face\picture_168_.png
D:\newfolder43_2\clear_face\picture_1691_.png
D:\newfolder43_2\clear_face\picture_169_.png
D:\newfolder43_2\clear_face\picture_16_.png
D:\newfolder43_2\clear_face\picture_170_.png
D:\newfolder43_2\clear_face\picture_171_.p

D:\newfolder43_2\clear_face\picture_201_.png
D:\newfolder43_2\clear_face\picture_2021_.png
D:\newfolder43_2\clear_face\picture_2028_.png
D:\newfolder43_2\clear_face\picture_202_.png
D:\newfolder43_2\clear_face\picture_203_.png
D:\newfolder43_2\clear_face\picture_2046_.png
D:\newfolder43_2\clear_face\picture_2051_.png
D:\newfolder43_2\clear_face\picture_2053_.png
D:\newfolder43_2\clear_face\picture_2059_.png
D:\newfolder43_2\clear_face\picture_2061_.png
D:\newfolder43_2\clear_face\picture_2062_.png
D:\newfolder43_2\clear_face\picture_2063_.png
D:\newfolder43_2\clear_face\picture_2064_.png
D:\newfolder43_2\clear_face\picture_2065_.png
D:\newfolder43_2\clear_face\picture_2066_.png
D:\newfolder43_2\clear_face\picture_2068_.png
D:\newfolder43_2\clear_face\picture_2077_.png
D:\newfolder43_2\clear_face\picture_2085_.png
D:\newfolder43_2\clear_face\picture_2088_.png
D:\newfolder43_2\clear_face\picture_209_.png
D:\newfolder43_2\clear_face\picture_20_.png
D:\newfolder43_2\clear_face\picture_2107

D:\newfolder43_2\clear_face\picture_2527_.png
D:\newfolder43_2\clear_face\picture_2529_.png
D:\newfolder43_2\clear_face\picture_252_.png
D:\newfolder43_2\clear_face\picture_2531_.png
D:\newfolder43_2\clear_face\picture_2533_.png
D:\newfolder43_2\clear_face\picture_2535_.png
D:\newfolder43_2\clear_face\picture_2537_.png
D:\newfolder43_2\clear_face\picture_2541_.png
D:\newfolder43_2\clear_face\picture_2547_.png
D:\newfolder43_2\clear_face\picture_2549_.png
D:\newfolder43_2\clear_face\picture_2550_.png
D:\newfolder43_2\clear_face\picture_2552_.png
D:\newfolder43_2\clear_face\picture_2559_.png
D:\newfolder43_2\clear_face\picture_2565_.png
D:\newfolder43_2\clear_face\picture_2578_.png
D:\newfolder43_2\clear_face\picture_2586_.png
D:\newfolder43_2\clear_face\picture_2587_.png
D:\newfolder43_2\clear_face\picture_2593_.png
D:\newfolder43_2\clear_face\picture_2595_.png
D:\newfolder43_2\clear_face\picture_2599_.png
D:\newfolder43_2\clear_face\picture_25_.png
D:\newfolder43_2\clear_face\picture_2

...

D:\newfolder43_2\fuzzy_face\picture_2228_.png
D:\newfolder43_2\fuzzy_face\picture_222_.png
D:\newfolder43_2\fuzzy_face\picture_2230_.png
D:\newfolder43_2\fuzzy_face\picture_2236_.png
D:\newfolder43_2\fuzzy_face\picture_2238_.png
D:\newfolder43_2\fuzzy_face\picture_223_.png
D:\newfolder43_2\fuzzy_face\picture_2242_.png
D:\newfolder43_2\fuzzy_face\picture_2246_.png
D:\newfolder43_2\fuzzy_face\picture_2248_.png
D:\newfolder43_2\fuzzy_face\picture_224_.png
D:\newfolder43_2\fuzzy_face\picture_2252_.png
D:\newfolder43_2\fuzzy_face\picture_2256_.png
D:\newfolder43_2\fuzzy_face\picture_225_.png
D:\newfolder43_2\fuzzy_face\picture_2262_.png
D:\newfolder43_2\fuzzy_face\picture_2268_.png
D:\newfolder43_2\fuzzy_face\picture_226_.png
D:\newfolder43_2\fuzzy_face\picture_2273_.png
D:\newfolder43_2\fuzzy_face\picture_2274_.png
D:\newfolder43_2\fuzzy_face\picture_2279_.png
D:\newfolder43_2\fuzzy_face\picture_227_.png
D:\newfolder43_2\fuzzy_face\picture_2285_.png
D:\newfolder43_2\fuzzy_face\picture_2288

D:\newfolder43_2\fuzzy_face\picture_2533_.png
D:\newfolder43_2\fuzzy_face\picture_2534_.png
D:\newfolder43_2\fuzzy_face\picture_2535_.png
D:\newfolder43_2\fuzzy_face\picture_2536_.png
D:\newfolder43_2\fuzzy_face\picture_2537_.png
D:\newfolder43_2\fuzzy_face\picture_2538_.png
D:\newfolder43_2\fuzzy_face\picture_2539_.png
D:\newfolder43_2\fuzzy_face\picture_253_.png
D:\newfolder43_2\fuzzy_face\picture_2540_.png
D:\newfolder43_2\fuzzy_face\picture_2541_.png
D:\newfolder43_2\fuzzy_face\picture_2542_.png
D:\newfolder43_2\fuzzy_face\picture_2543_.png
D:\newfolder43_2\fuzzy_face\picture_2544_.png
D:\newfolder43_2\fuzzy_face\picture_2545_.png
D:\newfolder43_2\fuzzy_face\picture_2546_.png
D:\newfolder43_2\fuzzy_face\picture_2547_.png
D:\newfolder43_2\fuzzy_face\picture_2548_.png
D:\newfolder43_2\fuzzy_face\picture_2549_.png
D:\newfolder43_2\fuzzy_face\picture_254_.png
D:\newfolder43_2\fuzzy_face\picture_2550_.png
D:\newfolder43_2\fuzzy_face\picture_2551_.png
D:\newfolder43_2\fuzzy_face\picture_

D:\newfolder43_2\fuzzy_face\picture_308_.png
D:\newfolder43_2\fuzzy_face\picture_3091_.png
D:\newfolder43_2\fuzzy_face\picture_3093_.png
D:\newfolder43_2\fuzzy_face\picture_3095_.png
D:\newfolder43_2\fuzzy_face\picture_3099_.png
D:\newfolder43_2\fuzzy_face\picture_309_.png
D:\newfolder43_2\fuzzy_face\picture_30_.png
D:\newfolder43_2\fuzzy_face\picture_3101_.png
D:\newfolder43_2\fuzzy_face\picture_3103_.png
D:\newfolder43_2\fuzzy_face\picture_3107_.png
D:\newfolder43_2\fuzzy_face\picture_310_.png
D:\newfolder43_2\fuzzy_face\picture_311_.png
D:\newfolder43_2\fuzzy_face\picture_3128_.png
D:\newfolder43_2\fuzzy_face\picture_312_.png
D:\newfolder43_2\fuzzy_face\picture_313_.png
D:\newfolder43_2\fuzzy_face\picture_314_.png
D:\newfolder43_2\fuzzy_face\picture_315_.png
D:\newfolder43_2\fuzzy_face\picture_316_.png
D:\newfolder43_2\fuzzy_face\picture_317_.png
D:\newfolder43_2\fuzzy_face\picture_318_.png
D:\newfolder43_2\fuzzy_face\picture_31_.png
D:\newfolder43_2\fuzzy_face\picture_3202_.png
D:\

D:\newfolder43_2\fuzzy_face\picture_44_.png
D:\newfolder43_2\fuzzy_face\picture_453_.png
D:\newfolder43_2\fuzzy_face\picture_454_.png
D:\newfolder43_2\fuzzy_face\picture_45_.png
D:\newfolder43_2\fuzzy_face\picture_46_.png
D:\newfolder43_2\fuzzy_face\picture_471_.png
D:\newfolder43_2\fuzzy_face\picture_472_.png
D:\newfolder43_2\fuzzy_face\picture_486_.png
D:\newfolder43_2\fuzzy_face\picture_488_.png
D:\newfolder43_2\fuzzy_face\picture_495_.png
D:\newfolder43_2\fuzzy_face\picture_497_.png
D:\newfolder43_2\fuzzy_face\picture_498_.png
D:\newfolder43_2\fuzzy_face\picture_49_.png
D:\newfolder43_2\fuzzy_face\picture_502_.png
D:\newfolder43_2\fuzzy_face\picture_503_.png
D:\newfolder43_2\fuzzy_face\picture_504_.png
D:\newfolder43_2\fuzzy_face\picture_505_.png
D:\newfolder43_2\fuzzy_face\picture_506_.png
D:\newfolder43_2\fuzzy_face\picture_50_.png
D:\newfolder43_2\fuzzy_face\picture_515_.png
D:\newfolder43_2\fuzzy_face\picture_51_.png
D:\newfolder43_2\fuzzy_face\picture_520_.png
D:\newfolder43_2

D:\newfolder43_2\fuzzy_face\picture_761_.png
D:\newfolder43_2\fuzzy_face\picture_762_.png
D:\newfolder43_2\fuzzy_face\picture_763_.png
D:\newfolder43_2\fuzzy_face\picture_764_.png
D:\newfolder43_2\fuzzy_face\picture_765_.png
D:\newfolder43_2\fuzzy_face\picture_766_.png
D:\newfolder43_2\fuzzy_face\picture_767_.png
D:\newfolder43_2\fuzzy_face\picture_768_.png
D:\newfolder43_2\fuzzy_face\picture_769_.png
D:\newfolder43_2\fuzzy_face\picture_770_.png
D:\newfolder43_2\fuzzy_face\picture_771_.png
D:\newfolder43_2\fuzzy_face\picture_772_.png
D:\newfolder43_2\fuzzy_face\picture_773_.png
D:\newfolder43_2\fuzzy_face\picture_774_.png
D:\newfolder43_2\fuzzy_face\picture_775_.png
D:\newfolder43_2\fuzzy_face\picture_776_.png
D:\newfolder43_2\fuzzy_face\picture_777_.png
D:\newfolder43_2\fuzzy_face\picture_779_.png
D:\newfolder43_2\fuzzy_face\picture_780_.png
D:\newfolder43_2\fuzzy_face\picture_781_.png
D:\newfolder43_2\fuzzy_face\picture_782_.png
D:\newfolder43_2\fuzzy_face\picture_783_.png
D:\newfold

In [15]:
img_2grad = np.zeros((28,30,30,3))
var_array_r = np.zeros((28))
var_array_g = np.zeros((28))
var_array_b = np.zeros((28))
var_array1 = np.zeros((28))
aver_array_r = np.zeros((28))
aver_array_g = np.zeros((28))
aver_array_b = np.zeros((28))
aver_array1 = np.zeros((28))
dir_path = "D:\\newfolder43_2"
save_path = "D:\\newfolder43_2_txt"
dirs = os.listdir(dir_path)
for i in range(len(dirs)):
    pic_names = os.listdir(dir_path+"\\"+dirs[i])
    for n in range(len(pic_names)):
        pic_path = dir_path+"\\"+dirs[i]+"\\"+pic_names[n]
        print(pic_path)
        #pic_path_gbk = pic_path.encode("gbk")
        #pic_path_ascii = pic_path_gbk.decode()
        #image = cv2.imread(pic_path_ascii)
        image=cv2.imdecode(np.fromfile(pic_path,dtype=np.uint8),-1)
        all_result_array = np.array([])
        all_var_array_r = np.array([])
        all_var_array_g = np.array([])
        all_var_array_b = np.array([])
        all_aver_array_r = np.array([])
        all_aver_array_g = np.array([])
        all_aver_array_b = np.array([])
        all_var_array1 = np.array([])
        all_aver_array1 = np.array([])
        for hor_num in range(4):
            for col_num in range(4):
                image1 = image[(30*hor_num):(30*(hor_num+1)),(30*col_num):(30*(col_num+1))]
                image_gaussion = cv2.filter2D(image1,cv2.CV_16S,kernel_)
                for m in range(28):
                    img_2grad[m] = cv2.filter2D(image_gaussion,cv2.CV_16S,p[m])
                    var_array_r[m] = img_2grad[m][:,:,0].std()
                    var_array_g[m] = img_2grad[m][:,:,1].std()
                    var_array_b[m] = img_2grad[m][:,:,2].std()
                    aver_array_r[m] = img_2grad[m][:,:,0].mean()
                    aver_array_g[m] = img_2grad[m][:,:,1].mean()
                    aver_array_b[m] = img_2grad[m][:,:,2].mean()
                    var_array1[m] = img_2grad[m].std()
                    aver_array1[m] = img_2grad[m].mean()
                
                all_var_array_r = np.hstack((all_var_array_r,var_array_r))
                all_var_array_g = np.hstack((all_var_array_g,var_array_g))
                all_var_array_b = np.hstack((all_var_array_b,var_array_b))
                all_aver_array_r = np.hstack((all_aver_array_r,aver_array_r))
                all_aver_array_g = np.hstack((all_aver_array_g,aver_array_g))
                all_aver_array_b = np.hstack((all_aver_array_b,aver_array_b))
                all_var_array1 = np.hstack((all_var_array1,var_array1))
                all_aver_array1 = np.hstack((all_aver_array1,aver_array1))
                
        all_var_array_r_max = all_var_array_r.max()
        all_var_array_r_min = all_var_array_r.min()
        all_var_array_g_max = all_var_array_g.max()
        all_var_array_g_min = all_var_array_g.min()
        all_var_array_b_max = all_var_array_b.max()
        all_var_array_b_min = all_var_array_b.min()
        
        all_aver_array_r_max = all_aver_array_r.max()
        all_aver_array_r_min = all_aver_array_r.min()
        all_aver_array_g_max = all_aver_array_g.max()
        all_aver_array_g_min = all_aver_array_g.min()
        all_aver_array_b_max = all_aver_array_b.max()
        all_aver_array_b_min = all_aver_array_b.min()
        
        all_var_array_max1 = all_var_array1.max()
        all_var_array_min1 = all_var_array1.min()
        all_aver_array_max1 = all_aver_array1.max()
        all_aver_array_min1 = all_aver_array1.min()
        
        all_var_array_r = np.true_divide((all_var_array_r-all_var_array_r_min),(all_var_array_r_max-all_var_array_r_min))
        all_var_array_g = np.true_divide((all_var_array_g-all_var_array_g_min),(all_var_array_g_max-all_var_array_g_min))
        all_var_array_b = np.true_divide((all_var_array_b-all_var_array_b_min),(all_var_array_b_max-all_var_array_b_min))
        all_aver_array_r = np.true_divide((all_aver_array_r-all_aver_array_r_min),(all_aver_array_r_max-all_aver_array_r_min))
        all_aver_array_g = np.true_divide((all_aver_array_g-all_aver_array_g_min),(all_aver_array_g_max-all_aver_array_g_min))
        all_aver_array_b = np.true_divide((all_aver_array_b-all_aver_array_b_min),(all_aver_array_b_max-all_aver_array_b_min))
        all_var_array1 = np.true_divide((all_var_array1-all_var_array_min1),(all_var_array_max1-all_var_array_min1))
        all_aver_array1 = np.true_divide((all_aver_array1-all_aver_array_min1),(all_aver_array_max1-all_aver_array_min1)) 
        
        all_var_array_r = np.true_divide((all_var_array_r-0.5),0.5) 
        all_var_array_g = np.true_divide((all_var_array_g-0.5),0.5) 
        all_var_array_b = np.true_divide((all_var_array_b-0.5),0.5) 
        all_aver_array_r = np.true_divide((all_aver_array_r-0.5),0.5)
        all_aver_array_g = np.true_divide((all_aver_array_g-0.5),0.5)
        all_aver_array_b = np.true_divide((all_aver_array_b-0.5),0.5)
        all_var_array1 = np.true_divide((all_var_array1-0.5),0.5) 
        all_aver_array1 = np.true_divide((all_aver_array1-0.5),0.5) 
                
        all_result_array = np.hstack((all_var_array_r,all_var_array_g,all_var_array_b,all_aver_array_r,all_aver_array_g,all_aver_array_b,all_var_array1,all_aver_array1))       
             
        name = save_path+"\\"+dirs[i]+"\\"+(pic_names[n].split("."))[0]+".txt"
        fw = open(name,"w")
        fw.write(','.join(str(num) for num in all_result_array))
        fw.close()

D:\newfolder43_2\clear_face\picture_1000_.png
D:\newfolder43_2\clear_face\picture_1002_.png
D:\newfolder43_2\clear_face\picture_1004_.png
D:\newfolder43_2\clear_face\picture_1005_.png
D:\newfolder43_2\clear_face\picture_1008_.png
D:\newfolder43_2\clear_face\picture_1009_.png
D:\newfolder43_2\clear_face\picture_100_.png
D:\newfolder43_2\clear_face\picture_1010_.png
D:\newfolder43_2\clear_face\picture_1013_.png
D:\newfolder43_2\clear_face\picture_1014_.png
D:\newfolder43_2\clear_face\picture_1015_.png
D:\newfolder43_2\clear_face\picture_1016_.png
D:\newfolder43_2\clear_face\picture_1018_.png
D:\newfolder43_2\clear_face\picture_1019_.png
D:\newfolder43_2\clear_face\picture_101_.png
D:\newfolder43_2\clear_face\picture_1020_.png
D:\newfolder43_2\clear_face\picture_1021_.png
D:\newfolder43_2\clear_face\picture_1024_.png
D:\newfolder43_2\clear_face\picture_1025_.png
D:\newfolder43_2\clear_face\picture_1027_.png
D:\newfolder43_2\clear_face\picture_1028_.png
D:\newfolder43_2\clear_face\picture_

D:\newfolder43_2\clear_face\picture_1326_.png
D:\newfolder43_2\clear_face\picture_1327_.png
D:\newfolder43_2\clear_face\picture_1328_.png
D:\newfolder43_2\clear_face\picture_1329_.png
D:\newfolder43_2\clear_face\picture_132_.png
D:\newfolder43_2\clear_face\picture_1330_.png
D:\newfolder43_2\clear_face\picture_1331_.png
D:\newfolder43_2\clear_face\picture_1332_.png
D:\newfolder43_2\clear_face\picture_1333_.png
D:\newfolder43_2\clear_face\picture_1334_.png
D:\newfolder43_2\clear_face\picture_1335_.png
D:\newfolder43_2\clear_face\picture_1337_.png
D:\newfolder43_2\clear_face\picture_1338_.png
D:\newfolder43_2\clear_face\picture_1339_.png
D:\newfolder43_2\clear_face\picture_1340_.png
D:\newfolder43_2\clear_face\picture_1341_.png
D:\newfolder43_2\clear_face\picture_1342_.png
D:\newfolder43_2\clear_face\picture_1343_.png
D:\newfolder43_2\clear_face\picture_1344_.png
D:\newfolder43_2\clear_face\picture_1345_.png
D:\newfolder43_2\clear_face\picture_1346_.png
D:\newfolder43_2\clear_face\picture

D:\newfolder43_2\clear_face\picture_1661_.png
D:\newfolder43_2\clear_face\picture_1663_.png
D:\newfolder43_2\clear_face\picture_1664_.png
D:\newfolder43_2\clear_face\picture_1666_.png
D:\newfolder43_2\clear_face\picture_1668_.png
D:\newfolder43_2\clear_face\picture_166_.png
D:\newfolder43_2\clear_face\picture_1671_.png
D:\newfolder43_2\clear_face\picture_1673_.png
D:\newfolder43_2\clear_face\picture_1676_.png
D:\newfolder43_2\clear_face\picture_167_.png
D:\newfolder43_2\clear_face\picture_1683_.png
D:\newfolder43_2\clear_face\picture_1684_.png
D:\newfolder43_2\clear_face\picture_1685_.png
D:\newfolder43_2\clear_face\picture_1686_.png
D:\newfolder43_2\clear_face\picture_1688_.png
D:\newfolder43_2\clear_face\picture_168_.png
D:\newfolder43_2\clear_face\picture_1691_.png
D:\newfolder43_2\clear_face\picture_169_.png
D:\newfolder43_2\clear_face\picture_16_.png
D:\newfolder43_2\clear_face\picture_170_.png
D:\newfolder43_2\clear_face\picture_171_.png
D:\newfolder43_2\clear_face\picture_173_.p

D:\newfolder43_2\clear_face\picture_2021_.png
D:\newfolder43_2\clear_face\picture_2028_.png
D:\newfolder43_2\clear_face\picture_202_.png
D:\newfolder43_2\clear_face\picture_203_.png
D:\newfolder43_2\clear_face\picture_2046_.png
D:\newfolder43_2\clear_face\picture_2051_.png
D:\newfolder43_2\clear_face\picture_2053_.png
D:\newfolder43_2\clear_face\picture_2059_.png
D:\newfolder43_2\clear_face\picture_2061_.png
D:\newfolder43_2\clear_face\picture_2062_.png
D:\newfolder43_2\clear_face\picture_2063_.png
D:\newfolder43_2\clear_face\picture_2064_.png
D:\newfolder43_2\clear_face\picture_2065_.png
D:\newfolder43_2\clear_face\picture_2066_.png
D:\newfolder43_2\clear_face\picture_2068_.png
D:\newfolder43_2\clear_face\picture_2077_.png
D:\newfolder43_2\clear_face\picture_2085_.png
D:\newfolder43_2\clear_face\picture_2088_.png
D:\newfolder43_2\clear_face\picture_209_.png
D:\newfolder43_2\clear_face\picture_20_.png
D:\newfolder43_2\clear_face\picture_2107_.png
D:\newfolder43_2\clear_face\picture_211

D:\newfolder43_2\clear_face\picture_2529_.png
D:\newfolder43_2\clear_face\picture_252_.png
D:\newfolder43_2\clear_face\picture_2531_.png
D:\newfolder43_2\clear_face\picture_2533_.png
D:\newfolder43_2\clear_face\picture_2535_.png
D:\newfolder43_2\clear_face\picture_2537_.png
D:\newfolder43_2\clear_face\picture_2541_.png
D:\newfolder43_2\clear_face\picture_2547_.png
D:\newfolder43_2\clear_face\picture_2549_.png
D:\newfolder43_2\clear_face\picture_2550_.png
D:\newfolder43_2\clear_face\picture_2552_.png
D:\newfolder43_2\clear_face\picture_2559_.png
D:\newfolder43_2\clear_face\picture_2565_.png
D:\newfolder43_2\clear_face\picture_2578_.png
D:\newfolder43_2\clear_face\picture_2586_.png
D:\newfolder43_2\clear_face\picture_2587_.png
D:\newfolder43_2\clear_face\picture_2593_.png
D:\newfolder43_2\clear_face\picture_2595_.png
D:\newfolder43_2\clear_face\picture_2599_.png
D:\newfolder43_2\clear_face\picture_25_.png
D:\newfolder43_2\clear_face\picture_2602_.png
D:\newfolder43_2\clear_face\picture_2

...

D:\newfolder43_2\fuzzy_face\picture_222_.png
D:\newfolder43_2\fuzzy_face\picture_2230_.png
D:\newfolder43_2\fuzzy_face\picture_2236_.png
D:\newfolder43_2\fuzzy_face\picture_2238_.png
D:\newfolder43_2\fuzzy_face\picture_223_.png
D:\newfolder43_2\fuzzy_face\picture_2242_.png
D:\newfolder43_2\fuzzy_face\picture_2246_.png
D:\newfolder43_2\fuzzy_face\picture_2248_.png
D:\newfolder43_2\fuzzy_face\picture_224_.png
D:\newfolder43_2\fuzzy_face\picture_2252_.png
D:\newfolder43_2\fuzzy_face\picture_2256_.png
D:\newfolder43_2\fuzzy_face\picture_225_.png
D:\newfolder43_2\fuzzy_face\picture_2262_.png
D:\newfolder43_2\fuzzy_face\picture_2268_.png
D:\newfolder43_2\fuzzy_face\picture_226_.png
D:\newfolder43_2\fuzzy_face\picture_2273_.png
D:\newfolder43_2\fuzzy_face\picture_2274_.png
D:\newfolder43_2\fuzzy_face\picture_2279_.png
D:\newfolder43_2\fuzzy_face\picture_227_.png
D:\newfolder43_2\fuzzy_face\picture_2285_.png
D:\newfolder43_2\fuzzy_face\picture_2288_.png
D:\newfolder43_2\fuzzy_face\picture_228_

D:\newfolder43_2\fuzzy_face\picture_2534_.png
D:\newfolder43_2\fuzzy_face\picture_2535_.png
D:\newfolder43_2\fuzzy_face\picture_2536_.png
D:\newfolder43_2\fuzzy_face\picture_2537_.png
D:\newfolder43_2\fuzzy_face\picture_2538_.png
D:\newfolder43_2\fuzzy_face\picture_2539_.png
D:\newfolder43_2\fuzzy_face\picture_253_.png
D:\newfolder43_2\fuzzy_face\picture_2540_.png
D:\newfolder43_2\fuzzy_face\picture_2541_.png
D:\newfolder43_2\fuzzy_face\picture_2542_.png
D:\newfolder43_2\fuzzy_face\picture_2543_.png
D:\newfolder43_2\fuzzy_face\picture_2544_.png
D:\newfolder43_2\fuzzy_face\picture_2545_.png
D:\newfolder43_2\fuzzy_face\picture_2546_.png
D:\newfolder43_2\fuzzy_face\picture_2547_.png
D:\newfolder43_2\fuzzy_face\picture_2548_.png
D:\newfolder43_2\fuzzy_face\picture_2549_.png
D:\newfolder43_2\fuzzy_face\picture_254_.png
D:\newfolder43_2\fuzzy_face\picture_2550_.png
D:\newfolder43_2\fuzzy_face\picture_2551_.png
D:\newfolder43_2\fuzzy_face\picture_2552_.png
D:\newfolder43_2\fuzzy_face\picture_

D:\newfolder43_2\fuzzy_face\picture_3093_.png
D:\newfolder43_2\fuzzy_face\picture_3095_.png
D:\newfolder43_2\fuzzy_face\picture_3099_.png
D:\newfolder43_2\fuzzy_face\picture_309_.png
D:\newfolder43_2\fuzzy_face\picture_30_.png
D:\newfolder43_2\fuzzy_face\picture_3101_.png
D:\newfolder43_2\fuzzy_face\picture_3103_.png
D:\newfolder43_2\fuzzy_face\picture_3107_.png
D:\newfolder43_2\fuzzy_face\picture_310_.png
D:\newfolder43_2\fuzzy_face\picture_311_.png
D:\newfolder43_2\fuzzy_face\picture_3128_.png
D:\newfolder43_2\fuzzy_face\picture_312_.png
D:\newfolder43_2\fuzzy_face\picture_313_.png
D:\newfolder43_2\fuzzy_face\picture_314_.png
D:\newfolder43_2\fuzzy_face\picture_315_.png
D:\newfolder43_2\fuzzy_face\picture_316_.png
D:\newfolder43_2\fuzzy_face\picture_317_.png
D:\newfolder43_2\fuzzy_face\picture_318_.png
D:\newfolder43_2\fuzzy_face\picture_31_.png
D:\newfolder43_2\fuzzy_face\picture_3202_.png
D:\newfolder43_2\fuzzy_face\picture_3203_.png
D:\newfolder43_2\fuzzy_face\picture_3204_.png
D:

D:\newfolder43_2\fuzzy_face\picture_45_.png
D:\newfolder43_2\fuzzy_face\picture_46_.png
D:\newfolder43_2\fuzzy_face\picture_471_.png
D:\newfolder43_2\fuzzy_face\picture_472_.png
D:\newfolder43_2\fuzzy_face\picture_486_.png
D:\newfolder43_2\fuzzy_face\picture_488_.png
D:\newfolder43_2\fuzzy_face\picture_495_.png
D:\newfolder43_2\fuzzy_face\picture_497_.png
D:\newfolder43_2\fuzzy_face\picture_498_.png
D:\newfolder43_2\fuzzy_face\picture_49_.png
D:\newfolder43_2\fuzzy_face\picture_502_.png
D:\newfolder43_2\fuzzy_face\picture_503_.png
D:\newfolder43_2\fuzzy_face\picture_504_.png
D:\newfolder43_2\fuzzy_face\picture_505_.png
D:\newfolder43_2\fuzzy_face\picture_506_.png
D:\newfolder43_2\fuzzy_face\picture_50_.png
D:\newfolder43_2\fuzzy_face\picture_515_.png
D:\newfolder43_2\fuzzy_face\picture_51_.png
D:\newfolder43_2\fuzzy_face\picture_520_.png
D:\newfolder43_2\fuzzy_face\picture_521_.png
D:\newfolder43_2\fuzzy_face\picture_523_.png
D:\newfolder43_2\fuzzy_face\picture_524_.png
D:\newfolder43_

D:\newfolder43_2\fuzzy_face\picture_764_.png
D:\newfolder43_2\fuzzy_face\picture_765_.png
D:\newfolder43_2\fuzzy_face\picture_766_.png
D:\newfolder43_2\fuzzy_face\picture_767_.png
D:\newfolder43_2\fuzzy_face\picture_768_.png
D:\newfolder43_2\fuzzy_face\picture_769_.png
D:\newfolder43_2\fuzzy_face\picture_770_.png
D:\newfolder43_2\fuzzy_face\picture_771_.png
D:\newfolder43_2\fuzzy_face\picture_772_.png
D:\newfolder43_2\fuzzy_face\picture_773_.png
D:\newfolder43_2\fuzzy_face\picture_774_.png
D:\newfolder43_2\fuzzy_face\picture_775_.png
D:\newfolder43_2\fuzzy_face\picture_776_.png
D:\newfolder43_2\fuzzy_face\picture_777_.png
D:\newfolder43_2\fuzzy_face\picture_779_.png
D:\newfolder43_2\fuzzy_face\picture_780_.png
D:\newfolder43_2\fuzzy_face\picture_781_.png
D:\newfolder43_2\fuzzy_face\picture_782_.png
D:\newfolder43_2\fuzzy_face\picture_783_.png
D:\newfolder43_2\fuzzy_face\picture_784_.png
D:\newfolder43_2\fuzzy_face\picture_785_.png
D:\newfolder43_2\fuzzy_face\picture_786_.png
D:\newfold

In [16]:
#这段测试代码，仍然可以用在txt为训练集的网络中，因为使用var_dataset()返回的也是224个数据，恰好和从txt文件中读回来的一样。
from PIL import Image
import shutil

model.cuda()
model.eval()

test_right_num = 0
test_right_num = t.Tensor([test_right_num]).int()
test_right_num = Variable(test_right_num)

test_dataset = txt_dataset("D:\\newfolder43_2_txt")
for img0, label0 in test_dataset:
    img0 = Variable(img0.float()).view(1,3584).cuda()
    label0 = Variable(t.Tensor([label0]).long()).cuda()
    output = model(img0)
    
    _,pred = t.max(output.cpu(),1)
    num_correct = (pred==label0.cpu()).sum()
    num_correct = num_correct.int()
    test_right_num = test_right_num + num_correct
    
acc = (float(test_right_num))/len(test_dataset)
print(test_right_num)
print(len(test_dataset))
print(acc)    
    

Variable containing:
 2735
[torch.IntTensor of size 1]

3172
0.8622320302648171
